<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Xavier", "transactions": [{"transaction-id": 837, "amount": 161}, {"transaction-id": 1599, "amount": 612}, {"transaction-id": 3013, "amount": 308}, {"transaction-id": 3107, "amount": 266}, {"transaction-id": 3632, "amount": 216}, {"transaction-id": 3980, "amount": 272}, {"transaction-id": 4633, "amount": 265}, {"transaction-id": 4974, "amount": -124}, {"transaction-id": 5590, "amount": 14}, {"transaction-id": 6305, "amount": 128}, {"transaction-id": 8113, "amount": 335}, {"transaction-id": 8860, "amount": -111}, {"transaction-id": 8981, "amount": 76}, {"transaction-id": 9318, "amount": 154}, {"transaction-id": 9530, "amount": 59}, {"transaction-id": 9791, "amount": 402}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Xavier", "transactions": [{"transaction-id": 837, "amount": 161}, {"transaction-id": 1599, "amount": 612}, {"transaction-id": 3013, "amount": 308}, {"transaction-id": 3107, "amount": 266}, {"transaction-id": 3632, "amount": 216}, {"transaction-id": 3980, "amount": 272}, {"transaction-id": 4633, "amount": 265}, {"transaction-id": 4974, "amount": -124}, {"transaction-id": 5590, "amount": 14}, {"transaction-id": 6305, "amount": 128}, {"transaction-id": 8113, "amount": 335}, {"transaction-id": 8860, "amount": -111}, {"transaction-id": 8981, "amount": 76}, {"transaction-id": 9318, "amount": 154}, {"transaction-id": 9530, "amount": 59}, {"transaction-id": 9791, "amount": 402}]}\n',
 '{"id": 1, "name": "Oliver", "transactions": [{"transaction-id": 143, "amount": 1452}, {"transaction-id": 206, "amount": 1359}, {"transaction-id": 378, "amount": 1355}, {"transaction-id": 493, "amount": 1363}, {"transaction-id": 579, "amount": 1274}, {"transaction-id": 595, "amount": 1400}, {

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Xavier',
  'transactions': [{'transaction-id': 837, 'amount': 161},
   {'transaction-id': 1599, 'amount': 612},
   {'transaction-id': 3013, 'amount': 308},
   {'transaction-id': 3107, 'amount': 266},
   {'transaction-id': 3632, 'amount': 216},
   {'transaction-id': 3980, 'amount': 272},
   {'transaction-id': 4633, 'amount': 265},
   {'transaction-id': 4974, 'amount': -124},
   {'transaction-id': 5590, 'amount': 14},
   {'transaction-id': 6305, 'amount': 128},
   {'transaction-id': 8113, 'amount': 335},
   {'transaction-id': 8860, 'amount': -111},
   {'transaction-id': 8981, 'amount': 76},
   {'transaction-id': 9318, 'amount': 154},
   {'transaction-id': 9530, 'amount': 59},
   {'transaction-id': 9791, 'amount': 402}]},
 {'id': 1,
  'name': 'Oliver',
  'transactions': [{'transaction-id': 143, 'amount': 1452},
   {'transaction-id': 206, 'amount': 1359},
   {'transaction-id': 378, 'amount': 1355},
   {'transaction-id': 493, 'amount': 1363},
   {'transaction-id': 579,

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 51,
  'name': 'Alice',
  'transactions': [{'transaction-id': 781, 'amount': 697},
   {'transaction-id': 1399, 'amount': 721},
   {'transaction-id': 1441, 'amount': 753},
   {'transaction-id': 1875, 'amount': 659},
   {'transaction-id': 1930, 'amount': 703},
   {'transaction-id': 2491, 'amount': 635},
   {'transaction-id': 2555, 'amount': 664},
   {'transaction-id': 2610, 'amount': 719},
   {'transaction-id': 2814, 'amount': 706},
   {'transaction-id': 3067, 'amount': 762},
   {'transaction-id': 3257, 'amount': 659},
   {'transaction-id': 3276, 'amount': 740},
   {'transaction-id': 3723, 'amount': 658},
   {'transaction-id': 3844, 'amount': 658},
   {'transaction-id': 3973, 'amount': 805},
   {'transaction-id': 4204, 'amount': 745},
   {'transaction-id': 4534, 'amount': 755},
   {'transaction-id': 4589, 'amount': 713},
   {'transaction-id': 4942, 'amount': 728},
   {'transaction-id': 5083, 'amount': 710},
   {'transaction-id': 5290, 'amount': 777},
   {'transaction-id': 5298, 'a

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 38},
 {'name': 'Alice', 'count': 27},
 {'name': 'Alice', 'count': 1},
 {'name': 'Alice', 'count': 44},
 {'name': 'Alice', 'count': 4})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(38, 27, 1, 44, 4)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

32.425891181988746

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 781, 'amount': 697},
  {'transaction-id': 1399, 'amount': 721},
  {'transaction-id': 1441, 'amount': 753},
  {'transaction-id': 1875, 'amount': 659},
  {'transaction-id': 1930, 'amount': 703},
  {'transaction-id': 2491, 'amount': 635},
  {'transaction-id': 2555, 'amount': 664},
  {'transaction-id': 2610, 'amount': 719},
  {'transaction-id': 2814, 'amount': 706},
  {'transaction-id': 3067, 'amount': 762},
  {'transaction-id': 3257, 'amount': 659},
  {'transaction-id': 3276, 'amount': 740},
  {'transaction-id': 3723, 'amount': 658},
  {'transaction-id': 3844, 'amount': 658},
  {'transaction-id': 3973, 'amount': 805},
  {'transaction-id': 4204, 'amount': 745},
  {'transaction-id': 4534, 'amount': 755},
  {'transaction-id': 4589, 'amount': 713},
  {'transaction-id': 4942, 'amount': 728},
  {'transaction-id': 5083, 'amount': 710},
  {'transaction-id': 5290, 'amount': 777},
  {'transaction-id': 5298, 'amount': 714},
  {'transaction-id': 5809, 'amount': 695},
  {'transact

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 781, 'amount': 697},
 {'transaction-id': 1399, 'amount': 721},
 {'transaction-id': 1441, 'amount': 753})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(697, 721, 753)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1091.2389631429728

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 127), ('Alice', 128), ('Alice', 129), ('Alice', 149), ('Bob', 156), ('Bob', 156), ('Bob', 156), ('Bob', 182), ('Charlie', 72), ('Charlie', 72), ('Charlie', 72), ('Charlie', 83), ('Dan', 108), ('Dan', 108), ('Dan', 108), ('Dan', 126), ('Edith', 60), ('Edith', 60), ('Edith', 60), ('Edith', 70), ('Frank', 144), ('Frank', 144), ('Frank', 144), ('Frank', 168), ('George', 132), ('George', 132), ('George', 132), ('George', 154), ('Hannah', 100), ('Hannah', 101), ('Hannah', 104), ('Hannah', 116), ('Ingrid', 107), ('Ingrid', 108), ('Ingrid', 108), ('Ingrid', 126), ('Jerry', 101), ('Jerry', 104), ('Jerry', 105), ('Jerry', 119), ('Kevin', 177), ('Kevin', 177), ('Kevin', 177), ('Kevin', 209), ('Laura', 60), ('Laura', 60), ('Laura', 60), ('Laura', 70), ('Michael', 48), ('Michael', 48), ('Michael', 48), ('Michael', 56), ('Norbert', 84), ('Norbert', 84), ('Norbert', 84), ('Norbert', 98), ('Oliver', 167), ('Oliver', 167), ('Oliver', 168), ('Oliver', 196), ('Patricia', 139), ('Patricia', 140

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 533), ('Bob', 650), ('Charlie', 299), ('Dan', 450), ('Edith', 250), ('Frank', 600), ('George', 550), ('Hannah', 421), ('Ingrid', 449), ('Jerry', 429), ('Kevin', 740), ('Laura', 250), ('Michael', 200), ('Norbert', 350), ('Oliver', 698), ('Patricia', 587), ('Quinn', 735), ('Ray', 547), ('Sarah', 554), ('Tim', 400), ('Ursula', 350), ('Victor', 277), ('Wendy', 300), ('Xavier', 623), ('Yvonne', 450), ('Zelda', 574)]
CPU times: user 125 ms, sys: 28.3 ms, total: 153 ms
Wall time: 506 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Xavier,"[{'transaction-id': 837, 'amount': 161}, {'tra..."
1,1,Oliver,"[{'transaction-id': 143, 'amount': 1452}, {'tr..."
2,2,Patricia,"[{'transaction-id': 228, 'amount': 200}, {'tra..."
3,4,Xavier,"[{'transaction-id': 210, 'amount': 160}, {'tra..."
4,5,Michael,"[{'transaction-id': 33, 'amount': 663}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 269 ms, sys: 11.7 ms, total: 280 ms
Wall time: 1.6 s


name
Alice      533
Bob        650
Charlie    299
Dan        450
Edith      250
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Xavier', 'amount': 161, 'transaction-id': 837},
 {'id': 0, 'name': 'Xavier', 'amount': 612, 'transaction-id': 1599},
 {'id': 0, 'name': 'Xavier', 'amount': 308, 'transaction-id': 3013})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Xavier,161,837
1,0,Xavier,612,1599
2,0,Xavier,308,3013
3,0,Xavier,266,3107
4,0,Xavier,216,3632


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 204 ms, sys: 24.6 ms, total: 229 ms
Wall time: 1.17 s


name
Alice       17283
Bob         25285
Charlie     10175
Dan         21706
Edith       17028
Frank       28714
George      23715
Hannah      15609
Ingrid       9727
Jerry       15783
Kevin       17540
Laura        7142
Michael      7074
Norbert     14168
Oliver      42339
Patricia    27288
Quinn       36969
Ray         21281
Sarah       19094
Tim         18391
Ursula      14880
Victor       5382
Wendy        6633
Xavier      29321
Yvonne      27335
Zelda       20138
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()